In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [8]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}).find("span",attrs={'class':'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'class':'a-price a-text-price'}).find("span", attrs={'class':'a-offscreen'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5 cm-cr-review-stars-spacing-big'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [9]:

if __name__ == '__main__':

    # add your user agent 
    # Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=mobiles&i=electronics&crid=3GUH7BWS28ZK&sprefix=mobile%2Celectronics%2C206&ref=nb_sb_noss_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [10]:
amazon_df

,title,price,rating,reviews,availability
0,"itel A70 Smartphone | 12GB* RAM, 256GB Storage...","₹7,299",Previous page,,In stock
1,"Redmi 13C (Stardust Black, 4GB RAM, 128GB Stor...","₹8,999",4.0 out of 5 stars,128 ratings,In stock
2,"Redmi 13C 5G (Starlight Black, 4GB RAM, 128GB ...","₹10,999",4.0 out of 5 stars,134 ratings,In stock
3,"realme narzo N55 (Prime Black, 6GB+128GB) 33W ...","₹9,499",4.1 out of 5 stars,"10,536 ratings",In stock
4,"Redmi 13C (Starshine Green, 4GB RAM, 128GB Sto...","₹8,999",4.0 out of 5 stars,128 ratings,In stock
5,"Redmi A2 (Sea Green, 2GB RAM, 64GB Storage)","₹5,499",3.9 out of 5 stars,"6,857 ratings",In stock
6,"Redmi A2 (Aqua Blue, 2GB RAM, 64GB Storage)","₹5,499",3.9 out of 5 stars,"6,857 ratings",In stock
7,"realme narzo 60X 5G（Nebula Purple 6GB,128GB St...","₹14,499",4.1 out of 5 stars,"5,879 ratings",In stock
8,"realme narzo N55 (Prime Blue, 6GB+128GB) 33W S...","₹9,499",4.1 out of 5 stars,"10,536 ratings",In stock
9,Nokia 105 Classic | Single SIM Keypad Phone wi...,₹999,3.6 out of 5 stars,314 ratings,In stock
